# Unit 2 Assignment: Building a Mixture of Experts (MoE) Router

In [30]:
pip install groq python-dotenv langchain langchain-groq

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.2 -> 26.0.1
[notice] To update, run: C:\Users\diyab\AppData\Local\Programs\Python\Python313\python.exe -m pip install --upgrade pip


In [31]:
import os
from dotenv import load_dotenv

load_dotenv()

groq_api_key = os.getenv("GROQ_API_KEY")

print("API Key Loaded:", groq_api_key is not None)

API Key Loaded: True


In [32]:
from langchain_groq import ChatGroq
router_llm = ChatGroq(
    model="llama-3.1-8b-instant",
    temperature=0
)

expert_llm = ChatGroq(
    model="llama-3.1-8b-instant",
    temperature=0.7
)

In [33]:
MODEL_CONFIG = {
    "technical": {
        "system_prompt": "You are a rigorous Technical Support Expert. You provide precise debugging advice, code snippets, and step-by-step solutions."
    },
    "billing": {
        "system_prompt": "You are an empathetic Billing Support Expert. You focus on refund policies, charges, and financial explanations."
    },
    "general": {
        "system_prompt": "You are a friendly General Support assistant helping with general inquiries."
    }
}

In [34]:
def route_prompt(user_input):
    router_prompt = [
        SystemMessage(
            content="""
You are a strict intent classifier.

Classify the user's message into ONE of these categories:

technical → programming errors, debugging, software issues
billing → payments, refunds, subscriptions
tool → real-time data requests like current price, weather, stock price, bitcoin price
general → greetings, casual chat, anything else

Return ONLY one word:
technical
billing
tool
general
"""
        ),
        HumanMessage(content=user_input)
    ]

    response = router_llm.invoke(router_prompt)
    return response.content.strip().lower()

In [35]:
print(route_prompt("My python script throws an IndexError"))
print(route_prompt("I was charged twice this month"))
print(route_prompt("Hello how are you"))

technical
billing
general


In [36]:
def process_request(user_input):
    category = route_prompt(user_input)

    if category == "tool":
        response = get_bitcoin_price()
        return {
            "category": category,
            "response": response
        }

    system_prompt = MODEL_CONFIG.get(category, MODEL_CONFIG["general"])["system_prompt"]

    messages = [
        SystemMessage(content=system_prompt),
        HumanMessage(content=user_input)
    ]

    final_response = expert_llm.invoke(messages).content

    return {
        "category": category,
        "response": final_response
    }

### Technical Expert

In [37]:
result = process_request("My python script is throwing an IndexError on line 5.")

print("Category:", result["category"])
print("\nResponse:\n")
print(result["response"])

Category: technical

Response:

To effectively troubleshoot the issue, I'll need more information. Please provide the following details:

1. **The code snippet**: Share the relevant lines of code around the IndexError (lines 1-7 would be helpful).
2. **The full error message**: Include the entire error message, including any error code or indices.
3. **Context**: Provide some context about the script, such as what it's trying to accomplish and any relevant libraries or frameworks being used.

With that information, I can help you identify the root cause of the IndexError and suggest a solution.

Assuming a generic scenario, here are some common reasons for an IndexError in Python:

1. **Out-of-bounds index**: You're trying to access an index that is outside the bounds of a list, tuple, or string.
2. **Missing or empty iterable**: You're trying to access an element from an empty or missing iterable.

Example code snippet:

```python
# Example list
my_list = [1, 2, 3]

# Attempting to ac

### Billing Expert

In [38]:
result = process_request("I was charged twice for my subscription this month.")

print("Category:", result["category"])
print("\nResponse:\n")
print(result["response"])

Category: billing

Response:

I'm so sorry to hear that you've been charged twice for your subscription this month. That can be frustrating and confusing. 

Can you please provide me with some details about your subscription and the charges? This will help me assist you more effectively. Here are some questions to get started:

1. What is the name of the subscription or service you're referring to?
2. What was the date you were charged the second time?
3. Did you receive any notifications or emails from us about the charge?
4. Have you checked your bank or credit card statement to confirm the second charge?

Once I have this information, I'll do my best to look into the matter and assist you in getting a refund for the duplicate charge.


### General Expert

In [39]:
result = process_request("Tell me a short motivational quote.")

print("Category:", result["category"])
print("\nResponse:\n")
print(result["response"])

Category: general

Response:

Here's a short motivational quote for you:

"Believe you can and you're halfway there."

- Theodore Roosevelt

I hope this inspires you to reach your goals!


### Bonus Challenge - Adding "Tool Use" expert 

In [40]:
def get_bitcoin_price():
    return "The current price of Bitcoin is $64,250."

In [41]:
result = process_request("What is the current price of Bitcoin?")

print("Category:", result["category"])
print("\nResponse:\n")
print(result["response"])

Category: tool

Response:

The current price of Bitcoin is $64,250.
